In [1]:
from pathlib import Path
import os
from joblib import Memory
from sklearn.datasets import load_svmlight_file
from scipy.sparse import csc_matrix
from scipy.special import expit
from scipy import sparse
from sklearn.preprocessing import normalize
from copy import copy
from sklearn import svm
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from scipy.stats import pearsonr

import pyarrow.feather as feather
import matplotlib.pyplot as plt
import plotnine as p9
import pandas as pd
import numpy as np

rand_state = 5991

In [2]:
# FUNCTIONS
def split_data(xx, yy, testsize = 1000):
    xtrain, xtest, y_train, y_test = train_test_split(xx,
                                                      yy,
                                                      test_size = testsize,
                                                      random_state = rand_state)
    xtrain, xvalid, y_train, y_valid = train_test_split(xtrain, y_train, 
                                                        test_size = testsize,
                                                        random_state = rand_state)
    y_train = np.array(y_train).reshape(-1, 1)
    y_test = np.array(y_test).reshape(-1, 1)
    y_valid = np.array(y_valid).reshape(-1, 1)

    print(" SHAPE of xtrain:", xtrain.shape)
    print("SHAPE of y_train:", y_train.shape)
    print("  SHAPE of xtest:", xtest.shape)
    print(" SHAPE of y_test:", y_test.shape)
    print(" SHAPE of xvalid:", xvalid.shape)
    print("SHAPE of y_valid:", y_valid.shape)

    return xtrain, y_train, xtest, y_test, xvalid, y_valid 

def get_acc_auc(y, p):
    acc = np.sum(y == p) / len(y)
    auc = roc_auc_score(y, p)
    return acc, auc

In [11]:
# PathsC:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/
idcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/idcsc.feather"
cipcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/cipcsc.feather"
geoloccsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/geoloccsc.feather"
instdemocsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/instdemocsc.feather"
studdemocsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/studdemocsc.feather"
pcipcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/pcipcsc.feather"
numcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/numcsc.feather"
tarcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/tarcsc.feather"
fulldf = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/scripts/postprocess/fulldf.feather"

In [12]:
id_csc = feather.read_feather(idcsc)
cip_csc = feather.read_feather(cipcsc)
geolocation_csc = feather.read_feather(geoloccsc)
inst_demographic_csc = feather.read_feather(instdemocsc)
stud_demographic_csc = feather.read_feather(studdemocsc)
pcip_csc = feather.read_feather(pcipcsc)
num_csc = feather.read_feather(numcsc)
tar_csc = feather.read_feather(tarcsc)

full_df = feather.read_feather(fulldf)

missing = pd.DataFrame(full_df.isna().sum())
missing.reset_index(inplace=True)
missing[missing[0] > 0]

,index,0


In [13]:
# Do the ids column in the target frame match the the ids column in the full frame for each unique UNITID?
df = pd.merge(tar_csc[["ids", "UNITID"]],
         full_df[["ids", "UNITID", "INSTNM"]],
         how = "left", left_on = "UNITID", right_on = "UNITID",
         suffixes=("_target", "_full"))
df["diff"] = df["ids_target"] - df["ids_full"]
df.loc[df["diff"] > 0 ]
# Yes

,ids_target,UNITID,ids_full,INSTNM,diff


In [ ]:
# Changing Majority Pell to have the value of 1 and Minority Pell to have the value of 0.
# check before
# df = pd.merge(id_csc, tar_csc, how = "left",
#               left_on = "UNITID", right_on = "UNITID",
#               suffixes = ["_id", "_target"])
# df[["PELLCAT_id", "PCTPELL", "PELLCAT_target"]]

# pc_dict = { 1: "MINPELL", 0: "MAJPELL" } # originally Minority Pell equals 1, and Majority Pell equals 0.
# pc_dict2 = { "MINPELL": 0, "MAJPELL": 1 } # I want to change Minority Pell to equal 0 and Majority Pell to equal 1.
# tar_csc["PELLCAT"] = tar_csc["PELLCAT"].map(pc_dict)
# tar_csc["PELLCAT"] = tar_csc["PELLCAT"].map(pc_dict2)
# id_csc["PELLCAT"] = id_csc["PELLCAT"].map(pc_dict)
# id_csc["PELLCAT"] = id_csc["PELLCAT"].map(pc_dict2)

# # check after
# df = pd.merge(id_csc, tar_csc, how = "left",
#               left_on = "UNITID", right_on = "UNITID",
#               suffixes = ["_id", "_target"])
# df[["PELLCAT_id", "PCTPELL", "PELLCAT_target"]]

In [ ]:
full_df.head(3)

,ids,UNITID,INSTNM,CIP01ASSOC,CIP01BACHL,CIP01CERT1,CIP01CERT2,CIP01CERT4,CIP03ASSOC,CIP03BACHL,...,NUM_ALL_UNDERGRADS_NOPROG_in2yr,NUM_ALL_UNDERGRADS_MAKEPROG_in2yr,NUM_COMPLETERS_FBR_in2yr,NUM_COMPLETERS_NOPROG_in2yr,DBRR1_PP_UG_BALANCE_REMAINING,DBRR4_PP_UG_BALANCE_REMAINING,DBRR5_PP_UG_BALANCE_REMAINING,DBRR1_PP_UG_AVG_OUTSTANDING_BALANCE_PERBORROWER,DBRR4_PP_UG_AVG_OUTSTANDING_BALANCE_PERBORROWER,DBRR5_PP_UG_AVG_OUTSTANDING_BALANCE_PERBORROWER
0,0,100654.0,Alabama A & M University,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,693.000000,113.000000,207.0000,298.000000,-2568428.0,-1815917.0,-1403097.0,20377.867804,16146.501497,13469.924157
1,1,100663.0,University of Alabama at Birmingham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1557.000001,1053.000002,417.0000,829.999999,-807333.0,6747.0,804701.0,21747.369253,15957.357009,12430.664875
2,2,100690.0,Amridge University,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,88.000000,69.350667,8.1229,10.972467,70898.0,359312.0,415266.0,12949.914179,9520.960938,8796.216418


In [14]:
ys = tar_csc["PELLCAT"].copy()
xs = full_df.drop(["ids", "UNITID"], axis = 1).copy()
print("SHAPE of xs:", xs.shape)
print("SHAPE of ys:", ys.shape)

SHAPE of xs: (5879, 1306)
SHAPE of ys: (5879,)


In [15]:
xtrain, y_train, xtest, y_test, xvalid, y_valid = split_data(xs, ys, testsize = 1000)

 SHAPE of xtrain: (3879, 1306)
SHAPE of y_train: (3879, 1)
  SHAPE of xtest: (1000, 1306)
 SHAPE of y_test: (1000, 1)
 SHAPE of xvalid: (1000, 1306)
SHAPE of y_valid: (1000, 1)


### Checking for non-linear associations between individual features and the binary target variable

In [17]:
print(xtrain.shape)

xnot = list(xtrain.columns[xtrain.columns.str.endswith("ASSOC") | xtrain.columns.str.endswith("BACHL") |
                      xtrain.columns.str.endswith("CERT1") | xtrain.columns.str.endswith("CERT2") |
                      xtrain.columns.str.endswith("CERT4")])
xnot2 = ["INSTNM", "ACCREDAGENCY", "ACCREDCODE", "CITY", "Season", "STABBR", "T4APPROVALDATE", "ZIP"]
xnot.extend(xnot2)

print("variables to be removed:")
print(len(xnot))
print("remaining variables:")

features = xtrain.columns.drop(xnot)

print(len(features))

missing = pd.DataFrame(xtrain[features].isna().sum())
missing.reset_index(inplace=True)
missing[missing[0] > 0]

(3879, 1306)
variables to be removed:
198
remaining variables:
1108


,index,0


In [18]:
scaler = StandardScaler()
poly = PolynomialFeatures(3)

scaler.fit(xtrain[features])
xtrain = pd.DataFrame(scaler.transform(xtrain[features]), columns = features)
xvalid = pd.DataFrame(scaler.transform(xvalid[features]), columns = features)
xtest = pd.DataFrame(scaler.transform(xtest[features]), columns = features)

log_reg = LogisticRegression(solver = "saga",
                             random_state = rand_state,
                             penalty = "l1",
                             class_weight = "balanced",
                             max_iter = 1000)

print("xtrain before: ", xtrain.shape)

xtrain before:  (3879, 1108)


In [19]:
# CREDIT: Dr. Vanderheyden wrote this code.
accuracies = []
for f in features:

    x = xtrain[f].values.reshape(-1, 1)
    y = y_train.reshape(-1, 1)
    ## LIN ##############################
    log_reg.fit(x, y)
    acc, auc = get_acc_auc(y, log_reg.predict(x))
    ## LOG #############################   
    xl = np.log(x - np.min(x) + 1)
    log_reg.fit(xl, y)
    lcc, luc = get_acc_auc(y, log_reg.predict(xl))
    if lcc / acc >= 1.1 or luc / auc >= 1.05:  # if bin accuracy is 110% of linear accuracy or ... AUC is 105% ...
        xtrain[f + " log"] = xl
        xvalid[f + " log"] = np.log(xvalid[f].values.reshape(-1, 1) - np.min(xtrain[f])+1)
        xtest[f + " log"] = np.log(xtest[f].values.reshape(-1, 1) - np.min(xtrain[f])+1)
    ## EXP #############################   
    xe = np.exp(x)
    log_reg.fit(xe, y)
    ecc, euc = get_acc_auc(y, log_reg.predict(xe))
    if ecc / acc >= 1.1 or euc / auc >= 1.05: 
        xtrain[f + " exp"] = xe
        xvalid[f + " exp"] = np.exp(xvalid[f].values.reshape(-1, 1))
        xtest[f + " exp"] = np.exp(xtest[f].values.reshape(-1, 1))
    ## POLY ############################# 
    poly.fit(x)
    xp = poly.transform(x)
    log_reg.fit(xp, y)
    pcc, puc = get_acc_auc(y, log_reg.predict(xp))
    if pcc / acc >= 1.1 or puc / auc >= 1.05:  # if bin accuracy is 110% of linear accuracy or ... AUC is 105% ...
        xtrain[f + " p2"] = x**2
        xtrain[f + " p3"] = x**3
        xvalid[f + " p2"] = (xvalid[f].values)**2
        xvalid[f + " p3"] = (xvalid[f].values)**3
        xtest[f + " p2"] = (xtest[f].values)**2
        xtest[f + " p3"] = (xtest[f].values)**3
    ## BIN #############################
    xmin = x.min()
    rnge = x.max() - xmin
    xtrn = 0 + ((x - xmin) > 1 * rnge / 10) + ((x - xmin) > 2 * rnge / 10) + ((x - xmin) > 3 * rnge / 10) + ( # the objects in each
                (x - xmin) > 4 * rnge / 10) + ((x - xmin) > 5 * rnge / 10) + ((x - xmin) > 6 * rnge / 10) + ( # bracket returns true
                (x - xmin) > 7 * rnge / 10) + ((x - xmin) > 8 * rnge / 10) + ((x - xmin) > 9 * rnge / 10)     # or false 
    xval = 0 + ((xvalid[f] - xmin) > 1 * rnge / 10) + ((xvalid[f] - xmin) > 2 * rnge / 10) + ((xvalid[f] - xmin) > 3 * rnge / 10) + (
                (xvalid[f] - xmin) > 4 * rnge / 10) + ((xvalid[f] - xmin) > 5 * rnge / 10) + ((xvalid[f] - xmin) > 6 * rnge / 10) + (
                (xvalid[f] - xmin) > 7 * rnge / 10) + ((xvalid[f] - xmin) > 8 * rnge / 10) + ((xvalid[f] - xmin) > 9 * rnge / 10)
    xtst = 0 + ((xtest[f] - xmin) > 1 * rnge / 10) + ((xtest[f] - xmin) > 2 * rnge / 10) + ((xtest[f] - xmin) > 5 * rnge / 10) + (
                (xtest[f] - xmin) > 3 * rnge / 10) + ((xtest[f] - xmin) > 4 * rnge / 10) + ((xtest[f] - xmin) > 6 * rnge / 10) + (
                (xtest[f] - xmin) > 7 * rnge / 10) + ((xtest[f] - xmin) > 8 * rnge / 10) + ((xtest[f] - xmin) > 9 * rnge / 10)
    encoder = TargetEncoder()
    encoder.fit(xtrn, y)
    xb = encoder.transform(xtrn)
    log_reg.fit(xb, y)
    bcc, buc = get_acc_auc(y, log_reg.predict(xb))
    if bcc / acc >= 1.1 or buc / auc >= 1.05: # if bin accuracy is 110% of linear accuracy or ... AUC is 105% ...
        xtrain[f + " Bin"] = xb
        xvalid[f + " Bin"] = encoder.transform(xval)
        xtest[f + " Bin"] = encoder.transform(xtst)
    ## COMPLETION #############################
    lDa = lcc / acc
    eDa = ecc / acc
    pDa = pcc / acc
    bDa = bcc / acc
    lda = luc / auc
    eda = euc / auc
    pda = puc / auc
    bda = buc / auc
    accuracies.append([f, acc, lcc, ecc, pcc, bcc, auc, luc, euc, puc, buc, lDa, eDa, pDa, bDa, lda, eda, pda, bda])
###############################################

colums = ["Feature","ACC: Linear", "ACC: Log", "ACC: Exp", "ACC: Poly3","ACC: Bin",
                    "AUC: Simple Linear", "AUC: Log", "AUC: Exp","AUC: Poly3", "AUC: Bin",
                    "ACC: LOG / Linear", "ACC: EXP / Linear", "ACC: Poly3 / Linear", "ACC: Bin / Linear",
                    "AUC: LOG / Linear", "AUC: EXP / Linear", "AUC: Poly3 / Linear", "AUC: Bin / Linear"]
accDf = pd.DataFrame(accuracies, columns = colums)
accDf

C:\Users\Nate\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Nate\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Nate\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Nate\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y t

,Feature,ACC: Linear,ACC: Log,ACC: Exp,ACC: Poly3,ACC: Bin,AUC: Simple Linear,AUC: Log,AUC: Exp,AUC: Poly3,AUC: Bin,ACC: LOG / Linear,ACC: EXP / Linear,ACC: Poly3 / Linear,ACC: Bin / Linear,AUC: LOG / Linear,AUC: EXP / Linear,AUC: Poly3 / Linear,AUC: Bin / Linear
0,CIP01ASSOC_encoded0,1662.325857,1662.325857,1662.325857,1662.325857,1662.325857,0.549289,0.549289,0.549289,0.549289,0.549289,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,CIP01ASSOC_encoded1,1658.758185,1658.758185,1658.758185,1658.758185,1658.758185,0.545723,0.545723,0.545723,0.545723,0.545723,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,CIP01ASSOC_encoded2,1613.567672,1613.567672,2269.000000,1613.567672,1613.567672,0.503566,0.503566,0.500000,0.503566,0.503566,1.000000,1.406201,1.000000,1.000000,1.000000,0.992919,1.000000,1.000000
3,CIP01BACHL_encoded0,1634.294148,1634.294148,1634.294148,1634.294148,1634.294148,0.515584,0.515584,0.515584,0.515584,0.515584,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,CIP01BACHL_encoded1,1632.595257,1632.595257,1632.595257,1632.595257,1632.595257,0.513380,0.513380,0.513380,0.513380,0.513380,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,DBRR4_PP_UG_BALANCE_REMAINING,1732.829853,1735.887858,2269.000000,1666.742975,1631.406032,0.596142,0.596392,0.500000,0.553957,0.520333,1.001765,1.309419,0.961862,0.941469,1.000419,0.838727,0.929236,0.872834
1104,DBRR5_PP_UG_BALANCE_REMAINING,1737.416860,1740.474865,2269.000000,1672.179428,1714.311936,0.600499,0.602872,0.500000,0.561008,0.583803,1.001760,1.305962,0.962451,0.986702,1.003953,0.832641,0.934237,0.972196
1105,DBRR1_PP_UG_AVG_OUTSTANDING_BALANCE_PERBORROWER,1795.518948,2043.557102,1750.158546,1798.067285,2078.894045,0.634981,0.620106,0.626052,0.636163,0.613911,1.138143,0.974737,1.001419,1.157824,0.976573,0.985937,1.001861,0.966816
1106,DBRR4_PP_UG_AVG_OUTSTANDING_BALANCE_PERBORROWER,1791.781387,1806.221964,2269.000000,1795.179170,2090.106729,0.620577,0.618602,0.500000,0.621268,0.591290,1.008059,1.266338,1.001896,1.166497,0.996817,0.805702,1.001113,0.952807


In [16]:
print(xtrain.shape)
accDf.head(5)
accDf.loc[:, ["ACC: LOG / Linear", "ACC: EXP / Linear", "ACC: Poly3 / Linear", "ACC: Bin / Linear",
           "AUC: LOG / Linear", "AUC: EXP / Linear", "AUC: Poly3 / Linear", "AUC: Bin / Linear"]].sort_values("ACC: LOG / Linear", ascending = False).head(5)
# accDf.sort_values("AUC: Log").head(20)

(3879, 1894)


,ACC: LOG / Linear,ACC: EXP / Linear,ACC: Poly3 / Linear,ACC: Bin / Linear,AUC: LOG / Linear,AUC: EXP / Linear,AUC: Poly3 / Linear,AUC: Bin / Linear
729,1.409317,1.048329,1.048329,1.000000,1.00000,1.000101,1.000101,1.00000
513,1.408871,1.408871,1.000000,1.000000,0.99974,0.999740,1.000000,1.00000
308,1.408574,1.408574,1.408574,0.999473,0.99992,0.999920,0.999920,0.99992
110,1.407831,1.407831,1.000000,1.000000,0.99878,0.998780,1.000000,1.00000
198,1.407090,1.407090,1.407090,1.000000,0.99976,0.999760,0.999760,1.00000


### Checking the Correlation of features

In [20]:
from scipy.stats import pearsonr

corrs = []
contFeat = list(xtrain.columns)
contFeat_length = len(contFeat)


for i in range(contFeat_length):
    for j in range(i + 1, contFeat_length):
        feati = xtrain[contFeat[i]].values.flatten()
        featj = xtrain[contFeat[j]].values.flatten()

        corr, _ = pearsonr(feati, featj)
        corrs.append([feati, featj, corr, contFeat[i], contFeat[j]])


C:\Users\Nate\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.


In [ ]:
correl = pd.DataFrame(corrs, columns = ["f1", "f2", "P Corr", "feat1", "feat2"])
################################
num_var_g50 = len(correl[abs(correl["P Corr"]) > 0.5])
pct_var_g50 = num_var_g50 / (contFeat_length * (contFeat_length - 1) / 2)
print(xtrain.shape)
print("Total number of features pairs: )", num_var_g50)
print("Number of features pairs with absolute Pearson Correl above 0.5: )", num_var_g50)
print("Percent of features pairs with absolute Pearson Correl above 0.5: )", pct_var_g50)

In [ ]:
pct_var_g50 * 100
correl[abs(correl["P Corr"]) > 0.5]

# Modelling

#### Logistic Regression

In [ ]:
from sklearn.metrics import roc_auc_score

log = LogisticRegression(solver = "saga",
                         random_state = rand_state, 
                         penalty = "l1", 
                         C = 0.05, 
                         class_weight = "balanced",
                         max_iter = 3000)
log.fit(xtrain, y_train)

Tprob = log.predict_proba(xtrain)
vprob = log.predict_proba(xvalid)
tprob = log.predict_proba(xtest)

print("Train AUC and Accuracy: ", roc_auc_score(y_train, Tprob[:,1]), 1 - np.mean(abs(y_train - log.predict(xtrain))))
print("Train AUC and Accuracy: ", roc_auc_score(y_test, tprob[:,1]), 1 - np.mean(abs(y_test - log.predict(xvalid))))
print("Val AUC and Accuracy: ", roc_auc_score(y_valid, vprob[:,1]), 1 - np.mean(abs(y_valid - log.predict(xtest))))
print("Number of Weights equal to zero: ", np.sum(log.coef_==0))

In [ ]:
logReg_lst = []
ccc = [100, 10, 1, 0.1, 0.001, 0.0001]
lll = np.array(range(10, 101, 10)) / 100

for cc in ccc:
    logReg = LogisticRegression(solver = "saga",
                                random_state = rand_state,
                                penalty = "l1",
                                C = cc,
                                class_weight = 'balanced',
                                max_iter = 100)
    logReg.fit(xtrain, y_train)

    Tpred = logReg.predict(xtrain)
    vpred = logReg.predict(xvalid)
    tpred = logReg.predict(xtest)

    Tprob = logReg.predict_proba(xtrain)
    vprob = logReg.predict_proba(xvalid)
    tprob = logReg.predict_proba(xtest)

    logReg_lst.append(["saga", "l1", cc, None, 100,
                roc_auc_score(y_train, Tprob[:,1]), 1 - np.mean(abs(y_train - Tpred)),
                roc_auc_score(y_test, tprob[:,1]), 1 - np.mean(abs(y_test - tpred)),
                roc_auc_score(y_valid, vprob[:,1]), 1 - np.mean(abs(y_valid- vpred)),
                np.sum(logReg.coef_==0)]
                )
for cc in ccc:
    logReg= LogisticRegression(solver = "saga",
                                random_state = rand_state,
                                penalty = "l2",
                                C = cc,
                                max_iter = 100)

    logReg.fit(xtrain, y_train)

    Tpred = logReg.predict(xtrain)
    vpred = logReg.predict(xvalid)
    tpred = logReg.predict(xtest)

    Tprob = logReg.predict_proba(xtrain)
    vprob = logReg.predict_proba(xvalid)
    tprob = logReg.predict_proba(xtest)

    logReg_lst.append(["saga", "l2", cc, None, 100,
                roc_auc_score(y_train, Tprob[:,1]), 1 - np.mean(abs(y_train - Tpred)),
                roc_auc_score(y_test, tprob[:,1]), 1 - np.mean(abs(y_test - tpred)),
                roc_auc_score(y_valid, vprob[:,1]), 1 - np.mean(abs(y_valid- vpred)),
                np.sum(logReg.coef_==0)]
                )
for cc in ccc:
    for ll in lll:
        logReg = LogisticRegression(solver = "saga",
                                    random_state = rand_state,
                                    penalty = "elasticnet",
                                    C = cc,
                                    l1_ratio = ll,
                                    max_iter = 100)

        logReg.fit(xtrain, y_train)

        Tpred = logReg.predict(xtrain)
        vpred = logReg.predict(xvalid)
        tpred = logReg.predict(xtest)

        Tprob = logReg.predict_proba(xtrain)
        vprob = logReg.predict_proba(xvalid)
        tprob = logReg.predict_proba(xtest)

        logReg_lst.append(["saga", "elasticnet", cc, ll, 100,
                    roc_auc_score(y_train, Tprob[:,1]), 1 - np.mean(abs(y_train - Tpred)),
                    roc_auc_score(y_test, tprob[:,1]), 1 - np.mean(abs(y_test - tpred)),
                    roc_auc_score(y_valid, vprob[:,1]), 1 - np.mean(abs(y_valid- vpred)),
                    np.sum(logReg.coef_==0)]
                    )
logReg_outcomes = pd.DataFrame(logReg_lst, columns = ["solver", "penalty", "C", "l1_ratio", "max_iter", "Train_AUC", "Train_Accurcy", "Test_AUC", "Test_Accurcy", "Valid_AUC", "Valid_Accurcy", "coef_eq_zero"])

logReg_outcomes.to_csv("logReg_out.csv")

#### Random Forest Classification

In [ ]:

from sklearn.ensemble import RandomForestClassifier

leafs = [1, 2, 3, 4, 5, 10, 20, 30, 40, 50]
depths = [5, 6, 7] # , 8, 9, 10, 11, 12

for l in leafs:
    for d in depths:
        forest = RandomForestClassifier(n_estimators = 1000,
                                        criterion = "entropy",
                                        max_depth = d,
                                        min_samples_leaf = l,
                                        class_weight = "balanced_subsample",
                                        random_state = 67)
        forest.fit(xtrain, y_train)

        Tprob = forest.predict_proba(xtrain)
        vprob = forest.predict_proba(xval)
        tprob = forest.predict_proba(xtest)

        print("Train AUC and Accuracy: ", l, d,roc_auc_score(y_train, Tprob[:,1]), 1 - np.mean(abs(y_train - forest.predict(xtrain))))
        print(" Test AUC and Accuracy: ", l, d, roc_auc_score(y_test, tprob[:,1]), 1 - np.mean(abs(y_test - forest.predict(xvalid))))
        print("  Val AUC and Accuracy: ", l, d,roc_auc_score(y_valid, vprob[:,1]), 1 - np.mean(abs(y_valid- forest.predict(xval))))
        print()  

#### Neural Network